In [1]:
import river
import river.datasets as datasets
from river import stream
from pprint import pprint
import os
from scipy.io import arff
import pandas as pd
from river import tree
from river import evaluate
from river import metrics

import numpy as np
from  streams.stream_section import StreamSection
from streams.utils import FL, FU, generate_stream_section
from my_datasets.getting_datasets import get_LED,get_Hyperplane,get_Electricity,get_AGRAWL,get_Airlines,get_CoverType,get_RandomRBF
from train_and_eval import train_and_evaluate
from functools import partial
from train_and_eval import train_and_evaluate
from river.metrics.accuracy import Accuracy
from multiprocessing.pool import ThreadPool
from river.datasets.synth import ConceptDriftStream
from tqdm import tqdm

# methods
from river.dummy import NoChangeClassifier,PriorClassifier
from river.cluster import CluStream
from river.forest.adaptive_random_forest import ARFClassifier
from river.tree.hoeffding_adaptive_tree_classifier import  HoeffdingAdaptiveTreeClassifier
from river.neighbors import KNNClassifier as riverKNN
from river.naive_bayes import GaussianNB
from semisupervised_methods.clustream_and_label import CluserAndLabel
from semisupervised_methods.incremental_classifier import IncrementalClassifer
from semisupervised_methods.one_nearest_neighbour import oneNNClassifer
from river.drift import ADWIN, PageHinkley


In [2]:
_ = [x for x,y in get_CoverType()]
_ = pd.DataFrame.from_records(_)
_

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,...,Soil_Type_30,Soil_Type_31,Soil_Type_32,Soil_Type_33,Soil_Type_34,Soil_Type_35,Soil_Type_36,Soil_Type_37,Soil_Type_38,Soil_Type_39
0,2596.0,51.0,3.0,258.0,0.0,510.0,221.0,232.0,148.0,6279.0,...,False,False,False,False,False,False,False,False,False,False
1,2590.0,56.0,2.0,212.0,-6.0,390.0,220.0,235.0,151.0,6225.0,...,False,False,False,False,False,False,False,False,False,False
2,2804.0,139.0,9.0,268.0,65.0,3180.0,234.0,238.0,135.0,6121.0,...,False,False,False,False,False,False,False,False,False,False
3,2785.0,155.0,18.0,242.0,118.0,3090.0,238.0,238.0,122.0,6211.0,...,False,False,False,False,False,False,False,False,False,False
4,2595.0,45.0,2.0,153.0,-1.0,391.0,220.0,234.0,150.0,6172.0,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
581007,2396.0,153.0,20.0,85.0,17.0,108.0,240.0,237.0,118.0,837.0,...,False,False,False,False,False,False,False,False,False,False
581008,2391.0,152.0,19.0,67.0,12.0,95.0,240.0,237.0,119.0,845.0,...,False,False,False,False,False,False,False,False,False,False
581009,2386.0,159.0,17.0,60.0,7.0,90.0,236.0,241.0,130.0,854.0,...,False,False,False,False,False,False,False,False,False,False
581010,2384.0,170.0,15.0,60.0,5.0,90.0,230.0,245.0,143.0,864.0,...,False,False,False,False,False,False,False,False,False,False


In [22]:
from streams.utils import FL, FU, generate_stream_section
from streams.stream_section import StreamSection
from river import metrics
from multiprocessing.pool import ThreadPool
from constants import NUMBER_OF_THREADS
import numpy as np
import logging
import datetime

def get_most_frequent(predictions):
    ''' 
    Get the most common predction

    Parameters
    ----------
    predictions: list
        The list of predicted labels

    Returns
    ---------
    int
        The most popular class

    '''
    return max(set(predictions), key=predictions.count)


def update_performance_measures(predictions, true_label, B, metrics):
    ''' 
    The prediction for each awaiting (for a label) instance is made every K iterations 

    Parameters
    ----------
    predictions: dict
        The dictionary of predctions 

    true_label: int
        The true class of an instance

    B: int
        Number of bins #TODO add somewhere the describtion

    metrics: dict
        The dictionary holding the metrics to calculate the results
    '''
    interval = (len(predictions)-1)/B
    preds = list(predictions.values())
    for b in range(B+2):
        if b == 0:
            metrics[b].update(true_label, preds[0])
        elif b == B+1:
            metrics[b].update(true_label, preds[-1])
        if preds[int(1+interval*(b-1)):int((1+interval*b))]: #interval not empty
                y_pred = get_most_frequent(
                    preds[int(1+interval*(b-1)):int((1+interval*b))])
                metrics[b].update(true_label, y_pred)


def make_prediction_for_awaiting(h, cur_idx, P, L, K):
    ''' 
    The prediction for each awaiting (for a label) instance is made every K iterations 

    Parameters
    ----------
    h: classifier
        The trained classifer that predicts the class

    cur_idx: int
        The index of a currently processed observation

    P: dict
        The dictionary with predictions 

    L: dict
        The dictionary with awaiting results

    K: int
        The number indicates how often the predction is made

    '''
    # foreach instance in L add predictionin time cur_idx in P
    for idx, instance in L.items():
        # make prediction every K instances
        if abs(idx-cur_idx) % K == 0:
            P[idx][cur_idx] = h.predict_one(instance)


def add_delay_constant(stream, delay, no_delete_period, dataset_name, q):
    new_stream = []
    i = 0
    for idx_1, idx_2, x, y, in stream:
        if i < no_delete_period:
            new_stream.append((i, i, x, y))
            i = i+1
            continue
        if (i-no_delete_period) % delay == 0 and i-no_delete_period != 0:
            i += delay
        new_stream.append((i, i, x, None))
        new_stream.append((i+delay, i, x, y))
        i += 1
    new_stream.sort(key=lambda x: x[0])
    return StreamSection(f'{dataset_name}_delay_{q[0]}_{q[1]}', new_stream, False)

def add_delay_random(stream, delay, no_delete_period, dataset_name, q):
    new_stream = []
    used_indexes = []
    i = 0
    for idx_1, idx_2, x, y, in stream:
        if i < no_delete_period:
            new_stream.append((i, i, x, y))
            i = i+1
            continue
        while i in used_indexes:
            i+=1
        delay = np.random.randint(1,20)
        while i+delay in used_indexes:
            delay+=1 # if sampling again infinite loop possible
        new_stream.append((i, i, x, None))
        new_stream.append((i+delay, i, x, y))
        used_indexes.extend([i,i+delay])
        i += 1
    new_stream.sort(key=lambda x: x[0])
    return StreamSection(f'{dataset_name}_delay_{q[0]}_{q[1]}', new_stream, False)



def train_for_stream( my_stream,methods, methods_params,methods_name,
                       metric_fun, K, B,warm_up_period):
    results = {}
    # how many labelled instances appered -> needed for prediction of awaiting examples 
    labelled_insances_cnt = 0
    logging.debug(f" Start processing {my_stream.__name__}")
    for mi, method in enumerate(methods):
            # initilaze method and variables
        m = method(**methods_params[mi])
        metrics = [metric_fun() for _ in range(B+2)]
        h = m
        L = {}
        P = {}
        logging.debug(f" Start processing method {methods_name[mi]}")
        for cur_idx, init_idx, x, y in my_stream.stream:
            # TODO: can it be in this place
           
            # unlabelled instance
            if y is None:
                # add instnace and index
                L[cur_idx] = x
                P[cur_idx] = {}
                P[cur_idx][cur_idx] = h.predict_one(x)
                # TODO: think what to do if the method cannot deal with unlabelled
                h = h.learn_one(x)

            # labelled instance
            else:
                labelled_insances_cnt+=1
                if cur_idx != init_idx and init_idx in P.keys():  # delayed label
                    P[init_idx][cur_idx] = h.predict_one(x)
                    L.pop(init_idx)

                    update_performance_measures(
                        P[init_idx], y, B, metrics)
                # TODO: probably need to implment a better option of evaluation-> for now test then train is used
                else: #TODO: dopytac sie czy tojest ok 
                    if m._timestamp > warm_up_period:  # if in warmup period the prediction cannot be made
                        # if it was not delayed only last prediction exists
                        metrics[B+1].update(y, h.predict_one(x))
                
                make_prediction_for_awaiting(h, labelled_insances_cnt, P, L, K) 
                h = h.learn_one(x, y)
        logging.info(f"{my_stream.__name__}, {methods_name[mi]}, {', '.join([str(t.get())for t in metrics])}")        
        results[methods_name[mi]] = metrics
    return my_stream.__name__,results


        


In [45]:
from river.preprocessing import StandardScaler


def generate_stream_section(dataset, stream_name, dataset_name, with_scaling = False, start=0, stop=1000):
    """
    generates the StreamSection 
    if synth geneartor for take the stop-start instances are taken
    if needed scaling added

    """
    #TODO: add exception
    # TODO: Maybe move to some globalk constants
    if dataset_name in ['LED', 'AGRAWL', 'RandomRBF','HyperPlane']:
        to_take = stop-start
        if not with_scaling:
            return StreamSection(stream_name, [(cur_idx, cur_idx, instance[0], instance[1]) for cur_idx, instance in enumerate(dataset.take(to_take))], True)
        else:
            scaler = StandardScaler()
            res = []
            for cur_index,(x,y) in enumerate(dataset.take(to_take)):
                scaler = scaler.learn_one(x)
                x = scaler.transform_one(x)
                res.append((cur_index,cur_index,x,y))
            return StreamSection(stream_name, res, True)
    elif dataset_name in ['Airlines', 'Cover_Type', 'Electricity']:
        # dataset pass as list
        if not with_scaling:
            return StreamSection(stream_name, [(cur_idx, cur_idx, instance[0], instance[1]) for cur_idx, instance in enumerate(dataset[start:stop])], True)
        else:
            scaler = StandardScaler()
            res = []
            for cur_index,(x,y) in  enumerate(dataset[start:stop]):
                scaler = scaler.learn_one(x)
                x = scaler.transform_one(x)
                res.append((cur_index,cur_index,x,y))
            return StreamSection(stream_name, res, True)
    else:
        to_take = stop-start
        return StreamSection(stream_name, [(cur_idx, cur_idx, instance[0], instance[1]) for cur_idx, instance in enumerate(dataset.take(to_take))], True)

In [13]:
initial_stream = generate_stream_section(cds,'initail_LED_Drift_gradual','initail_LED_Drift_gradual',start=0,stop=100000)

In [33]:
import datetime
from multiprocessing.pool import ThreadPool

from mgr.train_and_eval import add_delay_constant, add_delay_random, train_for_stream
from constants import NUMBER_OF_THREADS
import numpy as np
import logging
import datetime

def generate_streams1(initial_stream, dataset_name, q, probas, delay_type, delay, warm_up_period):
    stream_set = []  # TODO chcek where this should be placed

    stream_set.append(initial_stream)
    if delay_type == 1:
        initial_stream = add_delay_constant(
            initial_stream.stream, delay, warm_up_period, dataset_name, q)
        dataset_name += '_constant_delay_initial'
        stream_set.append(initial_stream)
    elif delay_type == 2:
        initial_stream = add_delay_random(
            initial_stream.stream, delay, warm_up_period, dataset_name, q)
        dataset_name += '_random_delay_initial'
        stream_set.append(initial_stream)

    for p in probas:
        ssl_stream = StreamSection(f'{dataset_name}_ssl_{p}_{q[0]}_{q[1]}', FU(
            initial_stream.stream, p, warm_up_period), False)
        lfs_stream = StreamSection(
            f'{dataset_name}_lfs_{p}_{q[0]}_{q[1]}', FL(ssl_stream.stream), True)
        stream_set.append(ssl_stream)
        stream_set.append(lfs_stream)
    return stream_set

def train_and_evaluate(initial_stream, dataset_name, Q, probas, methods, methods_params,methods_name,
                       metric_fun, delay_type, K, B, delay,warm_up_period=10):
    '''
    Main evaluation and traing function
    delay_type - 0 - NONE, 1 - equal, 2- random
    '''

    now = datetime.datetime.now()
    date_time = now.strftime("%d_%m_%Y_%H_%M_%S")
    logging.basicConfig(filename=f'logs\\{date_time}.log', filemode='w', format='%(asctime)s - %(message)s',level=logging.INFO,datefmt='%d-%b-%y %H:%M:%S')
    pool = ThreadPool(NUMBER_OF_THREADS)
    results = {}
    for q in Q:
        # preparing streams part
        stream_set = generate_streams1(
            initial_stream, dataset_name, q, probas, delay_type, delay,warm_up_period)
        logging.debug('TStreams generated')
        # train and evaluation part
        for my_stream in stream_set:
            results_for_q = train_for_stream( my_stream,methods, methods_params, methods_name,
                       metric_fun, K, B,warm_up_period)
    
            
        results[q] = results_for_q
    return results


In [46]:
initial_stream = generate_stream_section(cds,'LED_Drift_gradual','LED_Drift_gradual',start=0,stop=1000)

In [42]:
led3 = get_LED(0.1,True,3)
led5 = get_LED(0.1,True,5)
led7 = get_LED(0.1,True,7)

cds = ConceptDriftStream(ConceptDriftStream(led3,led5,position=25000,width = 5000),ConceptDriftStream(led5,led7,position=25000,width = 5000), position=50000,width=5000)

In [51]:
initial_stream

In [61]:
train_and_evaluate(initial_stream,dataset_name,Q,probas,methods,m_params,['HT'],Accuracy,2,B=50,K=10,delay=100)

KeyboardInterrupt: 

In [9]:
dataset_name = 'LED_Drift_gradual'
Q=[(0,100000)]
probas = [0.1]

HT = HoeffdingAdaptiveTreeClassifier
incClasif = IncrementalClassifer
m_params = [
 {'threshold':0.7,
  'classifier':HT,
  'params':{'grace_period':10,'max_depth':4}}]
methods = [incClasif]

In [14]:
res = train_and_evaluate(initial_stream,dataset_name,Q,probas,methods,m_params,['HT'],Accuracy,2,B=50,K=10,delay=1000)

In [19]:
os.listdir('logs')

['27_03_2023_19_55_43.log',
 '27_03_2023_20_25_57.log',
 '27_03_2023_20_27_51.log',
 '27_03_2023_20_28_36.log',
 '27_03_2023_20_33_53.log',
 '27_03_2023_22_04_08.log',
 '27_03_2023_22_09_33.log',
 '27_03_2023_22_11_40.log',
 '29_03_2023_21_45_40.log',
 'date_time.log']

In [55]:
rr = pd.read_csv('logs//29_03_2023_21_45_40.log',header=None)

In [58]:
rr

,0,1,2,3,4,5,6,7,8,9,...,44,45,46,47,48,49,50,51,52,53
0,29-Mar-23 22:14:54 - initail_LED_Drift_gradual,HT,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.669044
1,29-Mar-23 22:18:15 - LED_Drift_gradual_random_...,HT,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.729671
2,29-Mar-23 22:31:19 - LED_Drift_gradual_delay_0...,HT,0.642454,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.642314,0.642314
3,30-Mar-23 10:35:36 - LED_Drift_gradual_random_...,HT,0.541819,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.542274,0.542274
4,30-Mar-23 17:42:37 - initail_LED_Drift_gradual,HT,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.637007
5,30-Mar-23 17:42:42 - LED_Drift_gradual_delay_0...,HT,0.554545,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.554545,0.554545
6,30-Mar-23 17:42:52 - LED_Drift_gradual_random_...,HT,0.506159,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.509518,0.509518
7,30-Mar-23 17:42:54 - LED_Drift_gradual_random_...,HT,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.630045


In [60]:
list(rr.loc[:,0])

['29-Mar-23 22:14:54 - initail_LED_Drift_gradual',
 '29-Mar-23 22:18:15 - LED_Drift_gradual_random_delay_lfs_0.1_0_100000',
 '29-Mar-23 22:31:19 - LED_Drift_gradual_delay_0_100000',
 '30-Mar-23 10:35:36 - LED_Drift_gradual_random_delay_ssl_0.1_0_100000',
 '30-Mar-23 17:42:37 - initail_LED_Drift_gradual',
 '30-Mar-23 17:42:42 - LED_Drift_gradual_delay_0_100000',
 '30-Mar-23 17:42:52 - LED_Drift_gradual_random_delay_ssl_0.1_0_100000',
 '30-Mar-23 17:42:54 - LED_Drift_gradual_random_delay_lfs_0.1_0_100000']